# A.I Hub Data Preprocessing

In [1]:
def dir_padding(i):
    if i < 10: return '000' + str(i)
    elif i < 100: return '00' + str(i)
    elif i < 1000: return '0' + str(i)
    elif i < 10000 : return str(i)

폴더 포맷 맞추는 함수

In [2]:
def file_padding(i):
    if i < 10: return '00000' + str(i)
    elif i < 100: return '0000' + str(i)
    elif i < 1000: return '000' + str(i)
    elif i < 10000: return '00' + str(i)
    elif i < 100000: return '0' + str(i)
    else: return str(i)

파일 포맷 맞추는 함수

In [3]:
def get_path(base_path, base_dir, mid_dir, filename):
    return base_path + base_dir + '/' + mid_dir + '/' + filename

A.I Hub 데이터 폴더 구조 계산해주는 함수

In [4]:
text2 = "o/ 근데 (70%)/(칠십 퍼센트)가 커 보이긴 하는데 (200)/(이백) 벌다 (140)/(백 사십) 벌면 빡셀걸? b/"
text3 = "근데 (3학년)/(삼 학년) 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?"
def filter_double_copied(sentence):
    new_sentence = str()
    flag = False
    
    for ch in sentence:
        if ch == '(' and flag == False: 
            flag = True
            continue
        if ch == '(' and flag == True:
            flag = False
            continue
        if ch != ')' and flag == False:
            new_sentence += ch
    return new_sentence

filter_double_copied(text2)

'o/ 근데 칠십 퍼센트가 커 보이긴 하는데 이백 벌다 백 사십 벌면 빡셀걸? b/'

(A) / (B) 일 때 B만 가져와주는 함수

In [5]:
def del_special_ch(sentence):
    SENTENCE_MARK = ['.', '?', ',', '!']
    NOISE = ['o', 'n', 'u', 'b', 'l']
    EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';']
    import re
    new_sentence = str()
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MARK:
            # o/, n/ 등 처리
            if idx + 1 < len(sentence) and ch in NOISE and sentence[idx+1] == '/': continue 
        if ch == '%': new_sentence += '퍼센트'
        elif ch == '#': new_sentence += '샾'
        elif ch not in EXCEPT: new_sentence += ch
    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence
del_special_ch("b/ 아/ 근@데$ 삼 학^년% 때- 까&지*는: [국가]장학=금 바+ 받으면서 다녔던 건가? 몇 % 할인인거지? c# 배워봤어?")

'아 근데 삼 학년퍼센트 때 까지는 국가장학금 바 받으면서 다녔던 건가? 몇 퍼센트 할인인거지? c샾 배워봤어?'

char 단위로 판단하여 특수문자 및 노이즈 필터링

In [6]:
def sentence_filter(raw_sentence):
    return del_special_ch(filter_double_copied(raw_sentence))
sentence_filter("근데 삼 학년 때 까지는 국가장학금 바+ 받으면서 다녔던 건가?")

'근데 삼 학년 때 까지는 국가장학금 바 받으면서 다녔던 건가?'

위에 정의한 2개의 함수를 호출하여 문장 필터링을 해주는 함수

## KsponSpeech -> KaiSpeech

In [10]:
%%time
import shutil
from tqdm import trange

folder_change_point = [124 * x for x in range(1,5)]
base_path = "E:/한국어 음성데이터/"
base_dir = ['KsponSpeech_0' + str(x) for x in range(1,6)]
mid_dir = ['KsponSpeech_' + str(dir_padding(x)) for x in range(1,624)]
base_dir_idx = 4
mid_dir_idx = 602
past_mid_dir_idx = -1
TOTAL_NUM = 622545
NEW_PATH = "E:/한국어 음성데이터/KaiSpeech/"
NEW_FNAME = "KaiSpeech_"

print('started...')
END_POINT = 199999 + 70064 + 63633 + 126490 + 42240 + 59000 + 21080 + 20330
for file_num in trange(END_POINT, TOTAL_NUM):
    if file_num % 1000 == 0 and file_num != 1: mid_dir_idx += 1
    if mid_dir_idx in folder_change_point and mid_dir_idx != past_mid_dir_idx:
        past_mid_dir_idx = mid_dir_idx
        base_dir_idx += 1
    path = get_path(base_path, base_dir[base_dir_idx], mid_dir[mid_dir_idx], 'KsponSpeech_' + file_padding(file_num))
    new_path = NEW_PATH + NEW_FNAME + file_padding(file_num)
    shutil.copy(path + ".pcm", new_path + '.pcm') # pcm 파일 복붙
    f = open(path + ".txt", 'r') 
    raw_sentence = f.readline()
    f.close()
    filtered_sentence = sentence_filter(raw_sentence)
    f = open(new_path + ".txt", 'w')
    f.write(filtered_sentence)
    f.close()
print('ended!!!')

started...


100%|██████████████████████████████████████████████████████████████████████████| 19709/19709 [1:09:46<00:00,  4.71it/s]


ended!!!
Wall time: 1h 9min 46s


## Create char label

In [4]:
%%time
import os
from tqdm import trange
import pandas as pd
EXCEPTION = ['(', ')', '/']
folder_change_point = [124 * x for x in range(1,5)]
base_path = "E:/한국어 음성데이터/"
base_dir = ['KsponSpeech_0' + str(x) for x in range(1,6)]
mid_dir = ['KsponSpeech_' + str(dir_padding(x)) for x in range(1,624)]

base_dir_idx = 0
mid_dir_idx = 0
past_mid_dir_idx = -1
label_list = list()
label_freq = list()
print('started...')
TOTAL_NUM = 622545
for file_num in trange(1, TOTAL_NUM):
    if file_num % 1000 == 1 and file_num != 1: mid_dir_idx += 1
    if mid_dir_idx in folder_change_point and mid_dir_idx != past_mid_dir_idx:
        past_mid_dir_idx = mid_dir_idx
        base_dir_idx += 1
    path = get_path(base_path, base_dir[base_dir_idx], mid_dir[mid_dir_idx], 'KsponSpeech_' + file_padding(file_num))
    file = open(path + "_new.txt", 'r')
    text = file.readline()
    for ch in text:
        if ch not in label_list:
            label_list.append(ch)
            label_freq.append(1)
        else:
            label_freq[label_list.index(ch)] += 1

label_freq, label_list = zip(*sorted(zip(label_freq, label_list), reverse=True))
print('ended!!!')

label = {'id':[], 'char':[], 'freq' :[]}
for idx, ch_freq in enumerate(zip(label_list, label_freq)):
    ch, freq = ch_freq
    label['id'].append(idx)
    label['char'].append(ch)
    label['freq'].append(freq)
df_label = pd.DataFrame(label)
df_label.to_csv('./aihub_labels.csv', encoding='cp949', index=False)

started...


100%|██████████████████████████████████████████████| 622544/622544 [14:15<00:00, 727.71it/s]


ended!!!
Wall time: 14min 17s


In [9]:
char = df_label['char']
for i in char:
    if i == '%': print("Shit!")

Shit!


In [4]:
from collections import Counter, defaultdict
def preprocess_label(sentence, dict_):
    for ch in sentence:
        if ch == ' ':continue
        dict_is_all_you_need[ch] += 1

    keys = dict_.keys()
    values = dict_.values()

    values, keys = zip(*sorted(zip(values, keys), reverse=True))

In [9]:
df_label.to_csv('./aihub_labels.csv', encoding='cp949', index=False)

In [16]:
import pandas as pd
label = {'id':[], 'char':[], 'freq' :[]}
for idx, ch_freq in enumerate(zip(label_list, label_freq)):
    ch, freq = ch_freq
    label['id'].append(idx)
    label['char'].append(ch)
    label['freq'].append(freq)
df_label = pd.DataFrame(label)
df_label.to_csv('./aihub_labels.csv', encoding='utf-8', index=False)

AttributeError: module 'pandas' has no attribute '_libs'

In [9]:
#import pickle

#data = {
#    'label_list': label_list,
#    'label_freq': label_freq
#}

# save
with open('data2.pickle', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

NameError: name 'data' is not defined

In [18]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)

In [27]:
label_list = data['label_list']

In [28]:
label_list

(' ',
 '.',
 '그',
 '이',
 '는',
 '가',
 '고',
 '어',
 '아',
 '거',
 '지',
 '데',
 '?',
 '나',
 '하',
 '다',
 '서',
 '에',
 '도',
 '게',
 '니',
 ',',
 '기',
 '은',
 '면',
 '야',
 '있',
 '을',
 '한',
 '까',
 '해',
 '리',
 '라',
 '래',
 '사',
 '근',
 '들',
 '안',
 '로',
 '일',
 '내',
 '보',
 '제',
 '같',
 '자',
 '만',
 '시',
 '런',
 '너',
 '대',
 '때',
 '되',
 '으',
 '뭐',
 '진',
 '를',
 '잖',
 '러',
 '인',
 '오',
 '무',
 '었',
 '구',
 '했',
 '수',
 '간',
 '애',
 '우',
 '요',
 '생',
 '마',
 '렇',
 '냥',
 '짜',
 '주',
 '없',
 '말',
 '학',
 '스',
 '더',
 '많',
 '원',
 '정',
 '겠',
 '먹',
 '여',
 '금',
 '든',
 '부',
 '할',
 '막',
 '전',
 '번',
 '좋',
 '랑',
 '네',
 '람',
 '약',
 '건',
 '각',
 '음',
 '좀',
 '알',
 '잘',
 '걸',
 '모',
 '것',
 '상',
 '려',
 '장',
 '히',
 '않',
 '맞',
 '던',
 '르',
 '교',
 '바',
 '드',
 '냐',
 '십',
 '날',
 '치',
 '비',
 '단',
 '동',
 '또',
 '못',
 '얘',
 '중',
 '저',
 '의',
 '난',
 '엄',
 '봤',
 '걔',
 '화',
 '싶',
 '갔',
 '았',
 '집',
 '왜',
 '계',
 '공',
 '긴',
 '신',
 '적',
 '연',
 '직',
 '실',
 '영',
 '미',
 '응',
 '봐',
 '분',
 '테',
 '년',
 '트',
 '문',
 '돼',
 '물',
 '예',
 '터',
 '세',
 '럼',
 '청',
 '차',
 '친'

In [22]:
data

{'label_list': (' ',
  '.',
  '그',
  '이',
  '는',
  '가',
  '고',
  '어',
  '아',
  '거',
  '지',
  '데',
  '?',
  '나',
  '하',
  '다',
  '서',
  '에',
  '도',
  '게',
  '니',
  ',',
  '기',
  '은',
  '면',
  '야',
  '있',
  '을',
  '한',
  '까',
  '해',
  '리',
  '라',
  '래',
  '사',
  '근',
  '들',
  '안',
  '로',
  '일',
  '내',
  '보',
  '제',
  '같',
  '자',
  '만',
  '시',
  '런',
  '너',
  '대',
  '때',
  '되',
  '으',
  '뭐',
  '진',
  '를',
  '잖',
  '러',
  '인',
  '오',
  '무',
  '었',
  '구',
  '했',
  '수',
  '간',
  '애',
  '우',
  '요',
  '생',
  '마',
  '렇',
  '냥',
  '짜',
  '주',
  '없',
  '말',
  '학',
  '스',
  '더',
  '많',
  '원',
  '정',
  '겠',
  '먹',
  '여',
  '금',
  '든',
  '부',
  '할',
  '막',
  '전',
  '번',
  '좋',
  '랑',
  '네',
  '람',
  '약',
  '건',
  '각',
  '음',
  '좀',
  '알',
  '잘',
  '걸',
  '모',
  '것',
  '상',
  '려',
  '장',
  '히',
  '않',
  '맞',
  '던',
  '르',
  '교',
  '바',
  '드',
  '냐',
  '십',
  '날',
  '치',
  '비',
  '단',
  '동',
  '또',
  '못',
  '얘',
  '중',
  '저',
  '의',
  '난',
  '엄',
  '봤',
  '걔',
  '화',
  '싶',
  '갔',
  '았',
  '집',
  '왜',

In [ ]:
NEW_PATH = "E:/한국어 음성데이터/aihub_dataset/"
f